In [ ]:
!pip install tensorflow
!pip install matplotlib

In [4]:
# นำเข้า TensorFlow และ Keras
import tensorflow as tf
from tensorflow import keras

# นำเข้าเฉพาะที่จำเป็นสำหรับการสร้างโมเดล
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout

# นำเข้าชุดข้อมูล MNIST และเครื่องมือช่วย
from keras.datasets import mnist
from keras.utils import to_categorical

# นำเข้าไลบรารีสำหรับการคำนวณและแสดงผล
import numpy as np
import matplotlib.pyplot as plt

print("ไลบรารีทั้งหมดถูกนำเข้าเรียบร้อยแล้ว")

ไลบรารีทั้งหมดถูกนำเข้าเรียบร้อยแล้ว


In [ ]:
# โหลดข้อมูล MNIST และแบ่งข้อมูลเป็น 3 ส่วน: train validation และ test
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# พิมพ์เพื่อตรวจสอบขนาดของข้อมูล
print("ขนาดของข้อมูลฝึกฝน (X_train):", X_train.shape)
print("ขนาดของป้ายกำกับฝึกฝน (y_train):", y_train.shape)
print("ขนาดของข้อมูลทดสอบ (X_test):", X_test.shape)
print("ขนาดของป้ายกำกับทดสอบ (y_test):", y_test.shape)

In [ ]:
# แสดงภาพตัวอย่าง 9 ภาพแรกจากชุดข้อมูลฝึกฝน
plt.figure(figsize=(8, 8))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(X_train[i], cmap='gray') # แสดงภาพแบบ Grayscale
    plt.title(f"Label: {y_train[i]}")
    plt.axis('off') # ปิดแกน X, Y
plt.tight_layout()
plt.show()

In [ ]:
# 1. ปรับรูปร่างข้อมูล
# เราใช้ Flatten layer ในโมเดลแทนการ reshape ด้วยตนเองในขั้นตอนนี้
# เพื่อให้โค้ดในส่วนสร้างโมเดลเข้าใจง่ายขึ้น

# 2. ทำ Normalization
# แปลงชนิดข้อมูลเป็น float32 เพื่อให้หารได้ และปรับค่าให้อยู่ในช่วง 0-1
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# 3. ทำ One-Hot Encoding สำหรับป้ายกำกับ
num_classes = 10 # จำนวนคลาสคือ 10 (ตัวเลข 0-9)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print("ตัวอย่างป้ายกำกับหลังทำ One-Hot Encoding (สำหรับภาพแรก):", y_train)

In [ ]:
# เริ่มสร้างโมเดลแบบเรียงลำดับ
model = Sequential()

# ชั้นที่ 1: Flatten Layer
model.add(Flatten(input_shape=(28, 28)))

# ชั้นที่ 2: Hidden Layer แรก
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4)) # ป้องกัน Overfitting

# ชั้นที่ 3: Hidden Layer ที่สอง
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

# ชั้นที่ 4: Output Layer
model.add(Dense(num_classes, activation='softmax'))

# แสดงสรุปสถาปัตยกรรมของโมเดล
model.summary()

In [18]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# กำหนดค่าพารามิเตอร์การฝึก
batch_size = 128
epochs = 15

# เริ่มการฝึกโมเดล เพิ่ม tensorboard 
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))


print("\nการฝึกโมเดลเสร็จสิ้น!")

In [ ]:
# ประเมินโมเดลด้วยข้อมูลทดสอบ
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {score[0]:.4f}')
print(f'Test Accuracy: {score[1]:.4f}') 

# แสดงกราฟการฝึก
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')    
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')          
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()  

In [ ]:
# ทำนายภาพทดสอบ
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)  
# แสดงผลการทำนายสำหรับภาพทดสอบ 9 ภาพแรก
plt.figure(figsize=(8, 8))
for i in range(25):     
    plt.subplot(5, 5, i + 1)
    plt.imshow(X_test[i], cmap='gray') # แสดงภาพแบบ Grayscale
    plt.title(f"Predicted: {predicted_classes[i]}\nActual: {np.argmax(y_test[i])}")
    plt.axis('off') # ปิดแกน X, Y
plt.tight_layout()
plt.show()  
